In [1]:
from collections import OrderedDict

import torch
import torch.nn.functional as F
import torchvision
from torch import nn
from torchvision.models._utils import IntermediateLayerGetter
from typing import Dict, List

import sys
sys.path.append('../')
from util.misc import NestedTensor, is_main_process

from models.position_encoding import build_position_encoding
from models.backbone import BackboneBase

import argparse
import datetime
import json
from models import build_model
from datasets import build_dataset
from datasets import get_coco_api_from_dataset
import util.misc as utils
from engine import evaluate, train_one_epoch
from models import build_model

from feed import LbpDataset

import pandas as pd
import numpy as np

In [2]:
from main import get_args_parser
parser = argparse.ArgumentParser('DETR training and evaluation script', parents=[get_args_parser()])
parser.add_argument('--workers', default=8, type=int)
args = parser.parse_args(args=[])
args.coco_path = '../COCO/'
args.batch_size = 4
args.resume = '../trained_models/r50_deformable_detr-checkpoint.pth'
args.distributed = False
args.output_dir = '../trained_models/'
args.num_class = 1 # remove back ground task, only real task
args.epochs = 100
args.lr_drop = 50
args.lr = 0.0002
args.device = 'cuda:0'

In [3]:
# dataset_train = build_dataset(image_set='train', args=args)
# dataset_val = build_dataset(image_set='val', args=args)
# images, targets = next(iter(dataset_train))
# print(type(dataset_val))
# # targets
# dataset_val

In [4]:
from feed import CLASS_MAPPER, get_train_test_list, train_transforms, val_transforms
df = pd.read_csv('../../data/df.csv')
df.label = df.label.apply(lambda x : CLASS_MAPPER[str(x)])
print(df.shape)
# Data loading code#
train_list, test_list = get_train_test_list(df)
train_dataset = LbpDataset(train_list, default_path='/home/beomgon/Dataset/scl/', transform=train_transforms)
test_dataset = LbpDataset(test_list, default_path='/home/beomgon/Dataset/scl/', transform=val_transforms)  

(9735, 12)


../feed.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xmax'] = df.apply(lambda x : x['xmin'] + x['w'], axis=1)
../feed.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ymax'] = df.apply(lambda x : x['ymin'] + x['h'], axis=1)
../feed.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

total 7176 train 5382 test 1794
5382
1794


In [5]:
train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.SequentialSampler(test_dataset)
data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=args.batch_size,
                                           shuffle=False,
                                           num_workers=args.workers,
                                           # pin_memory=True,
                                           sampler=train_sampler,
                                           collate_fn=utils.collate_fn)    

data_loader_test = torch.utils.data.DataLoader(
    test_dataset, batch_size=args.batch_size,
    sampler=test_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)


In [6]:
images, targets = next(iter(data_loader))
images.tensors.shape

torch.Size([4, 3, 1024, 1024])

In [7]:
# lr_backbone_names = ["backbone.0", "backbone.neck", "input_proj", "transformer.encoder"]
def match_name_keywords(n, name_keywords):
    out = False
    for b in name_keywords:
        if b in n:
            out = True
            break
    return out

In [8]:
device = torch.device(args.device)
model, criterion, postprocessors = build_model(args)

model_without_ddp = model
n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('number of params:', n_parameters)
    
param_dicts = [
    {
        "params":
            [p for n, p in model_without_ddp.named_parameters()
             if not match_name_keywords(n, args.lr_backbone_names) and not match_name_keywords(n, args.lr_linear_proj_names) and p.requires_grad],
        "lr": args.lr,
    },
    {
        "params": [p for n, p in model_without_ddp.named_parameters() if match_name_keywords(n, args.lr_backbone_names) and p.requires_grad],
        "lr": args.lr_backbone,
    },
    {
        "params": [p for n, p in model_without_ddp.named_parameters() if match_name_keywords(n, args.lr_linear_proj_names) and p.requires_grad],
        "lr": args.lr * args.lr_linear_proj_mult,
    }
]
optimizer = torch.optim.AdamW(param_dicts, lr=args.lr,
                              weight_decay=args.weight_decay)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, args.lr_drop)    

number of params: 39847265


In [9]:
# model_without_ddp

In [10]:
checkpoint = torch.load(args.resume, map_location='cpu')
checkpoint.keys()

dict_keys(['model', 'optimizer', 'lr_scheduler', 'epoch', 'args'])

In [11]:
# if args.sgd:
#     optimizer = torch.optim.SGD(param_dicts, lr=args.lr, momentum=0.9,
#                                 weight_decay=args.weight_decay)
# else:
#     optimizer = torch.optim.AdamW(param_dicts, lr=args.lr,
#                                   weight_decay=args.weight_decay)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, args.lr_drop)

In [12]:
if args.resume:
    if args.resume.startswith('https'):
        checkpoint = torch.hub.load_state_dict_from_url(
            args.resume, map_location='cpu', check_hash=True)
    else:
        checkpoint = torch.load(args.resume, map_location='cpu')
    print('pretrained model is used')
    model_without_ddp.load_state_dict(checkpoint['model'])
        
class_embed = nn.Linear(in_features=256, out_features=args.num_class + 1, bias=True)         
model_without_ddp.class_embed = nn.ModuleList([class_embed for _ in range(6)])

model_without_ddp.to(device)
print('model is loaded to gpu')

pretrained model is used
model is loaded to gpu


In [13]:
model.class_embed

ModuleList(
  (0): Linear(in_features=256, out_features=2, bias=True)
  (1): Linear(in_features=256, out_features=2, bias=True)
  (2): Linear(in_features=256, out_features=2, bias=True)
  (3): Linear(in_features=256, out_features=2, bias=True)
  (4): Linear(in_features=256, out_features=2, bias=True)
  (5): Linear(in_features=256, out_features=2, bias=True)
)

In [14]:
from engine import train_one_epoch
import time
from pathlib import Path
import os

output_dir = Path(args.output_dir)

print("Start training")
start_time = time.time()
for epoch in range(args.start_epoch, args.epochs):
    if args.distributed:
        sampler_train.set_epoch(epoch)
    train_stats = train_one_epoch(model_without_ddp, criterion, data_loader, optimizer, device, epoch, args.clip_max_norm)
    lr_scheduler.step()
    
    if epoch > 40 and epoch % 5 == 0 :
        if args.output_dir:
            checkpoint = {
                'model': model_without_ddp.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'args': args,
                'epoch': epoch
            }
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'model_{}.pth'.format(epoch)))
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'checkpoint.pth'))

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

Start training
Epoch: [0]  [   0/1346]  eta: 0:30:05  lr: 0.000200  class_error: 40.00  grad_norm: 4554.39  loss: 355.0887 (355.0887)  loss_ce: 58.7201 (58.7201)  loss_bbox: 0.2462 (0.2462)  loss_giou: 0.8388 (0.8388)  loss_ce_0: 58.6806 (58.6806)  loss_bbox_0: 0.2634 (0.2634)  loss_giou_0: 0.8154 (0.8154)  loss_ce_1: 57.6101 (57.6101)  loss_bbox_1: 0.2862 (0.2862)  loss_giou_1: 0.8897 (0.8897)  loss_ce_2: 61.5981 (61.5981)  loss_bbox_2: 0.2714 (0.2714)  loss_giou_2: 0.8841 (0.8841)  loss_ce_3: 55.1966 (55.1966)  loss_bbox_3: 0.2569 (0.2569)  loss_giou_3: 0.8799 (0.8799)  loss_ce_4: 56.5336 (56.5336)  loss_bbox_4: 0.2526 (0.2526)  loss_giou_4: 0.8649 (0.8649)  loss_ce_unscaled: 29.3601 (29.3601)  class_error_unscaled: 40.0000 (40.0000)  loss_bbox_unscaled: 0.0492 (0.0492)  loss_giou_unscaled: 0.4194 (0.4194)  cardinality_error_unscaled: 22.0000 (22.0000)  loss_ce_0_unscaled: 29.3403 (29.3403)  loss_bbox_0_unscaled: 0.0527 (0.0527)  loss_giou_0_unscaled: 0.4077 (0.4077)  cardinality_err